In [29]:
import pyodbc
import json
import os
from dotenv import load_dotenv
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

In [30]:
# Load the collections data from JSON file
with open ('nguondulieu\collections.json', 'r') as file:
    collections_data = json.load(file)

df_collection = pd.DataFrame(collections_data)

df_collection

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6072\3155150588.py:2: SyntaxWarning: invalid escape sequence '\c'
  with open ('nguondulieu\collections.json', 'r') as file:


,collection,name,owner,contracts_address
0,boredapeyachtclub,Bored Ape Yacht Club,0x0000000000000000000000000000000000000000,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d
1,pudgypenguins,Pudgy Penguins,0xf54c9a0e44a5f5afd27c7ac8a176a843b9114f1d,0xbd3531da5cf5857e7cfaa92426877b022e612cf8
2,cryptopunks,CryptoPunks,0xc352b534e8b987e036a93539fd6897f53488e56a,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb
3,milady,Milady Maker,0x43e2e78c8e6f2ce57fbd970c811e70e0044389aa,0x5af0d9827e0c53e4799bb226655a1de152a425a5
4,mutant-ape-yacht-club,Mutant Ape Yacht Club,0xa858ddc0445d8131dac4d1de01f834ffcba52ef1,0x60e4d786628fea6478f785a6d7e704777c86a7c6
5,memelandcaptainz,Memeland Captainz,0x1c593c369136264abfb908b4b20e74c9a5949417,0x769272677fab02575e84945f03eca517acc544cc
6,lilpudgys,Lil Pudgys,0xf54c9a0e44a5f5afd27c7ac8a176a843b9114f1d,0x524cab2ec69124574082676e6f654a18df49a048
7,remilio-babies,Redacted Remilio Babies,0x43e2e78c8e6f2ce57fbd970c811e70e0044389aa,0xd3d9ddd0cf0a5f0bfb8f7fceae075df687eaebab
8,azuki,Azuki,0x2ae6b0630ebb4d155c6e04fcb16840ffa77760aa,0xed5af388653567af2f388e6224dc7c4b3241c544
9,matr1x-kuku,MATR1X KUKU,0x6ff4dab3dbe9e7eb5264fcb11f3194ed9bf40f7e,0x596a5cd859ad53fec23cd3fcd77522f0b407920d


In [31]:
# kiểm tra những giá trị null
missing_value = df_collection.isna()
for column in missing_value.columns.values.tolist():
    print(missing_value[column].value_counts())

collection
False    20
Name: count, dtype: int64
name
False    20
Name: count, dtype: int64
owner
False    20
Name: count, dtype: int64
contracts_address
False    20
Name: count, dtype: int64


In [32]:
# in ra những dòng duplacate
duplicate_indices = df_collection.duplicated(keep=False)
duplicate_rows = df_collection[duplicate_indices]
duplicate_rows

,collection,name,owner,contracts_address
9,matr1x-kuku,MATR1X KUKU,0x6ff4dab3dbe9e7eb5264fcb11f3194ed9bf40f7e,0x596a5cd859ad53fec23cd3fcd77522f0b407920d
10,matr1x-kuku,MATR1X KUKU,0x6ff4dab3dbe9e7eb5264fcb11f3194ed9bf40f7e,0x596a5cd859ad53fec23cd3fcd77522f0b407920d


In [33]:
# xóa những dòng duplicate 
df_collection.drop_duplicates(subset=['collection'], keep='first', inplace=True)
print(len(df_collection))

19


In [34]:
# load môi trường
load_dotenv()
server = os.getenv('SERVER')
database = os.getenv('DATABASE')

# kiểm tra biến có tồn tại
if server is None:
    raise EnvironmentError("Environment variable 'SERVER' is not set.")
if database is None:
    raise EnvironmentError("Environment variable 'DATABASE' is not set.")

try:
    # tạo kết nối
    connection = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + 
        ';DATABASE=' + database + 
        ';Trusted_Connection=yes;'
    )

    cursor = connection.cursor()

    # định nghĩa tên bảng
    table_name = 'collections'

    # câu tạo bnagr
    create_table_query = f'''
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{table_name}' AND xtype='U')
    CREATE TABLE {table_name} (
        collection_slug VARCHAR(50) PRIMARY KEY,
        name VARCHAR(50),
        owner VARCHAR(50),
        contracts_address VARCHAR(50)
    )
    '''

    cursor.execute(create_table_query)
    connection.commit()

    # câu lệnh insert
    merge_query = f'''
    MERGE INTO {table_name} AS target
    USING (VALUES (?, ?, ?, ?)) AS source (collection_slug, name, owner, contracts_address)
    ON target.collection_slug = source.collection_slug
    WHEN MATCHED THEN 
        UPDATE SET name = source.name, owner = source.owner, contracts_address = source.contracts_address
    WHEN NOT MATCHED THEN
        INSERT (collection_slug, name, owner, contracts_address)
        VALUES (source.collection_slug, source.name, source.owner, source.contracts_address);
    '''

    # Insert dữ liệu
    for index, row in df_collection.iterrows():
        cursor.execute(merge_query, row['collection'], row['name'], row['owner'], row['contracts_address'])
        connection.commit()

    # đống kết nối
    cursor.close()
    connection.close()

except pyodbc.Error as ex:
    print("An error occurred while connecting to the database:", ex)
